# Setup

In [ ]:
!pip install scipy==1.6.3
!pip install scikit_optimize==0.8.1
!pip install scikit_learn==0.24.2
! git clone https://github.com/zykhoo/predicting_hamiltonian_dynamics.git

     |████████████████████████████████| 27.4 MB 1.3 MB/s 
  Attempting uninstall: scipy
    Found existing installation: scipy 1.4.1
    Uninstalling scipy-1.4.1:
      Successfully uninstalled scipy-1.4.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
albumentations 0.1.12 requires imgaug<0.2.7,>=0.2.5, but you have imgaug 0.2.9 which is incompatible.
     |████████████████████████████████| 101 kB 4.3 MB/s 
     |████████████████████████████████| 22.3 MB 61.8 MB/s 
  Attempting uninstall: scikit-learn
    Found existing installation: scikit-learn 1.0.2
    Uninstalling scikit-learn-1.0.2:
      Successfully uninstalled scikit-learn-1.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
yellowbrick 1.4 requires scikit-learn>=1.0.0, but 

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Run all

In [ ]:
import numpy as np

experiment,sys,dim = "NN","pendulum",2
f1 = lambda x: x[1]
f2 = lambda x: -np.sin(x[0])
H = lambda x: 1/2*x[1]**2+(1-np.cos(x[0]))
spacedim=[(-2*np.pi, 2*np.pi), (-1.2, 1.2)]
fvector = lambda x: np.asarray([f1(x), f2(x)])
h= 0.1
x0, H0 = 0.,0.
initialcon = [64, 128, 256, 512, 1024]
LR=0.001
diagdist = np.sum(np.square(np.asarray([spacedim[0][0], spacedim[0][1]]), np.asarray([spacedim[1][0], spacedim[1][1]])))
epsilon = 0.1

"""# Test dataset creation"""

from predicting_hamiltonian_dynamics import groundtruth_2dim
from tqdm import tqdm
import time 

evaluation_length_long = 50
evaluation_length_one = 1

xxlong,yylong = np.linspace(spacedim[0][0], spacedim[0][1], 5), np.linspace(spacedim[1][0], spacedim[1][1], 5)
xlong,ylong = np.meshgrid(xxlong,yylong)
xxshort,yyshort = np.linspace(spacedim[0][0], spacedim[0][1], 20), np.linspace(spacedim[1][0], spacedim[1][1], 20)
xshort,yshort = np.meshgrid(xxshort,yyshort)

trajectories_groundtruth_start_long = np.expand_dims(groundtruth_2dim.classicTrajectory(np.asarray([[0.4],[0.]]),f1,f2,h = 0.1,N=evaluation_length_long,n_h = 1),0)
for i in tqdm(np.expand_dims(np.c_[np.ravel(xlong),np.ravel(ylong)],2)):
  results_200 = np.expand_dims(groundtruth_2dim.classicTrajectory(i,f1,f2,h = 0.1,N=evaluation_length_long,n_h = 1),0)
  trajectories_groundtruth_start_long = np.vstack((trajectories_groundtruth_start_long, results_200))
trajectories_groundtruth_start_long.shape # output no., start and final, p and q, full traj

trajectories_groundtruth_start_short = np.expand_dims(groundtruth_2dim.classicTrajectory(np.asarray([[0.4],[0.]]),f1,f2,h = 0.1,N=evaluation_length_one,n_h = 1),0)
for i in tqdm(np.expand_dims(np.c_[np.ravel(xshort),np.ravel(yshort)],2)):
  results_200 = np.expand_dims(groundtruth_2dim.classicTrajectory(i,f1,f2,h = 0.1,N=evaluation_length_one,n_h = 1),0)
  trajectories_groundtruth_start_short = np.vstack((trajectories_groundtruth_start_short, results_200))
trajectories_groundtruth_start_short.shape # output no., start and final, p and q, full traj

within_array = groundtruth_2dim.get_within_array(trajectories_groundtruth_start_long, spacedim)

100%|██████████| 400/400 [00:00<00:00, 1784.42it/s]


In [ ]:
path = '/content/drive/MyDrive/SSI/Baseline v2/synthetic systems (upload)/'

In [ ]:
from predicting_hamiltonian_dynamics.models import NN_2dim
from predicting_hamiltonian_dynamics.models import GP_2dim
from predicting_hamiltonian_dynamics.models import PINN_2dim
from predicting_hamiltonian_dynamics.models import PIGP_2dim


for i in range(20):
  seed = i
  np.random.seed(seed=seed)
  for ini in initialcon: 

    start, final = groundtruth_2dim.CreateTrainingDataTrajClassicIntRandom(1,ini,spacedim,h,f1,f2,seed = seed,n_h = 1)

    delta = start.copy()
    delta[0,:] = f1(start)
    delta[1,:] = f2(start)

    """# NN"""

    from predicting_hamiltonian_dynamics.models import NN_2dim
    import torch

    if torch.cuda.is_available():
      device=torch.device('cuda')
    else:
      device=torch.device('cpu')
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)


    wholemat, evalmat = NN_2dim.data_preprocessing(start, delta, device)

    import torch.optim as optim
    import time 

    net = NN_2dim.Net(dim,16,dim)
    starttime = time.time() 
    net = NN_2dim.train(net, wholemat, evalmat, optimizer=optim.Adam(net.parameters(), lr=LR), batchsize=10, iter=1600, )
    traintime = time.time()-starttime

    from tqdm import tqdm
    from predicting_hamiltonian_dynamics.models import NN_2dim
    from predicting_hamiltonian_dynamics import metrics


    MSE_long, time_long, MSE_long_naive, time_long_naive, MSE_within, time_within, MSE_within_naive, time_within_naive, MSE_onestep, time_onestep, MSE_vectorfield, time_vectorfield, _, _ = NN_2dim.compute_metrics_NN(net, h, diagdist, xshort, yshort, xlong, ylong, evaluation_length_long, within_array, trajectories_groundtruth_start_long, trajectories_groundtruth_start_short, fvector)

    file_object = open(path + 'Pendulum_random.txt', 'a')
    file_object.write('NN, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s \n' %(ini, seed, traintime, MSE_long, time_long, MSE_long_naive, time_long_naive, MSE_within, time_within, MSE_within_naive, time_within_naive, MSE_onestep, time_onestep, MSE_vectorfield, time_vectorfield))
    file_object.close()


    """# GP"""

    from sklearn.gaussian_process import GaussianProcessRegressor
    from sklearn.gaussian_process.kernels import RBF
    from sklearn import preprocessing

    output = delta
    scaler = preprocessing.MinMaxScaler()
    input = scaler.fit_transform(start.transpose())
    kernel = 1 * RBF(length_scale=2., length_scale_bounds="fixed")
    gaussian_process = GaussianProcessRegressor(kernel=kernel, n_restarts_optimizer=0, optimizer = None, random_state = seed)
    starttime = time.time() 
    gaussian_process.fit(input, output.transpose())
    traintime = time.time()-starttime
    gaussian_process.predict(scaler.transform([[0.7,0.8]]))

    MSE_long, time_long, MSE_long_naive, time_long_naive, MSE_within, time_within, MSE_within_naive, time_within_naive, MSE_onestep, time_onestep, MSE_vectorfield, time_vectorfield, _, _ = GP_2dim.compute_metrics_GP(gaussian_process, scaler, h, diagdist, xshort, yshort, xlong, ylong, evaluation_length_long, within_array, trajectories_groundtruth_start_long, trajectories_groundtruth_start_short, fvector)

    file_object = open(path + 'Pendulum_random.txt', 'a')
    file_object.write('GP, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s \n' %(ini, seed, traintime, MSE_long, time_long, MSE_long_naive, time_long_naive, MSE_within, time_within, MSE_within_naive, time_within_naive, MSE_onestep, time_onestep, MSE_vectorfield, time_vectorfield))
    file_object.close()


    """# PINN"""

    from predicting_hamiltonian_dynamics.models import PINN_2dim
    import torch

    if torch.cuda.is_available():
      device=torch.device('cuda')
    else:
      device=torch.device('cpu')
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)

    wholemat, evalmat = PINN_2dim.data_preprocessing(start, delta,device)
    net=PINN_2dim.Net(dim,16,1)
    starttime = time.time()
    net,avg_vallosses,avg_lossli,avg_f1li,avg_f2li,avg_f3li,avg_f4li=PINN_2dim.train(net,bs=10,num_epoch=5000,initial_conditions=initialcon,device=device,
                                                                                    wholemat=wholemat,evalmat=evalmat,x0=x0,H0=H0,dim=dim,LR=LR,patience=500,c1=1,c2=1,c3=1,c4=1)
    traintime = time.time()-starttime

    MSE_long, time_long, MSE_long_naive, time_long_naive, MSE_within, time_within, MSE_within_naive, time_within_naive, MSE_onestep, time_onestep, MSE_vectorfield, time_vectorfield, _, _ = PINN_2dim.compute_metrics_PINN(net, device, h, diagdist, xshort, yshort, xlong, ylong, evaluation_length_long, within_array, trajectories_groundtruth_start_long, trajectories_groundtruth_start_short, fvector)
    
    file_object = open(path + 'Pendulum_random.txt', 'a')
    file_object.write('PINN, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s \n' %(ini, seed, traintime, MSE_long, time_long, MSE_long_naive, time_long_naive, MSE_within, time_within, MSE_within_naive, time_within_naive, MSE_onestep, time_onestep, MSE_vectorfield, time_vectorfield))
    file_object.close()

    """# PIGP"""

    from predicting_hamiltonian_dynamics.models import PIGP_2dim
    GP = PIGP_2dim.BertalanGP()
    starttime = time.time()
    GP.train(start,delta,h)
    traintime = time.time()-starttime

    MSE_long, time_long, MSE_long_naive, time_long_naive, MSE_within, time_within, MSE_within_naive, time_within_naive, MSE_onestep, time_onestep, MSE_vectorfield, time_vectorfield, _, _ = PIGP_2dim.compute_metrics_PIGP(GP, h, diagdist, xshort, yshort, xlong, ylong, evaluation_length_long, within_array, trajectories_groundtruth_start_long, trajectories_groundtruth_start_short, fvector)

    file_object = open(path + 'Pendulum_random.txt', 'a')
    file_object.write('PIGP, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s \n' %(ini, seed, traintime, MSE_long, time_long, MSE_long_naive, time_long_naive, MSE_within, time_within, MSE_within_naive, time_within_naive, MSE_onestep, time_onestep, MSE_vectorfield, time_vectorfield))
    file_object.close()


training loss tensor(0.7501, dtype=torch.float64, grad_fn=<MeanBackward0>)
validation loss tensor(0.7043, dtype=torch.float64, grad_fn=<MeanBackward0>)
training loss tensor(0.1927, dtype=torch.float64, grad_fn=<MeanBackward0>)
validation loss tensor(0.1400, dtype=torch.float64, grad_fn=<MeanBackward0>)
training loss tensor(0.1653, dtype=torch.float64, grad_fn=<MeanBackward0>)
validation loss tensor(0.1305, dtype=torch.float64, grad_fn=<MeanBackward0>)
training loss tensor(0.1037, dtype=torch.float64, grad_fn=<MeanBackward0>)
validation loss tensor(0.1041, dtype=torch.float64, grad_fn=<MeanBackward0>)
training loss tensor(0.0733, dtype=torch.float64, grad_fn=<MeanBackward0>)
validation loss tensor(0.0696, dtype=torch.float64, grad_fn=<MeanBackward0>)
training loss tensor(0.0583, dtype=torch.float64, grad_fn=<MeanBackward0>)
validation loss tensor(0.0538, dtype=torch.float64, grad_fn=<MeanBackward0>)
training loss tensor(0.0497, dtype=torch.float64, grad_fn=<MeanBackward0>)
validation lo

  0%|          | 0/50 [00:00<?, ?it/s]/content/predicting_hamiltonian_dynamics/models/NN_2dim.py:25: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at  ../torch/csrc/utils/tensor_new.cpp:201.)
  fstage = lambda stg: h * torch.squeeze(net(torch.tensor([q+stg,p]).float().transpose(1,0)),0).detach().numpy().transpose()[:dim]
100%|██████████| 50/50 [00:00<00:00, 174.80it/s]

100%|██████████| 50/50 [00:00<00:00, 1235.57it/s]

100%|██████████| 50/50 [00:00<00:00, 180.91it/s]

100%|██████████| 50/50 [00:00<00:00, 206.76it/s]

100%|██████████| 50/50 [00:00<00:00, 100.20it/s]

100%|██████████| 50/50 [00:00<00:00, 365.62it/s]

100%|██████████| 50/50 [00:00<00:00, 3107.90it/s]

100%|██████████| 34/34 [00:00<00:00, 481.37it/s]

100%|██████████| 50/50 [00:00<00:00, 434.37it/s]

100%|██████████| 50/50 [00:00<00:00, 499.16it/s]

100%|██

 
epoch= 0  time= 0.031059980392456055  loss= 1.026474267244339  val_loss= 2.2710102796554565  f1= 0.47867648309251515  f2= 0.4252674871088704  f3= 0.1206264557937781  f4= 0.0019038199298651555 percent lr= 0.001
 
epoch= 10  time= 0.2207343578338623  loss= 0.9366083045800527  val_loss= 2.036981761455536  f1= 0.4920728067609857  f2= 0.44290291078526955  f3= 1.4055694952238204e-05  f4= 0.0016185129133820426 percent lr= 0.001
 
epoch= 20  time= 0.3979191780090332  loss= 0.9670887490113577  val_loss= 1.9582027792930603  f1= 0.440524394795776  f2= 0.5247431457883515  f3= 3.0245978753858555e-05  f4= 0.001790986420934568 percent lr= 0.001
 
epoch= 30  time= 0.5701496601104736  loss= 0.8086269696553549  val_loss= 1.8178406953811646  f1= 0.4210030492350732  f2= 0.3825801159361351  f3= 0.0002918682500118545  f4= 0.004751910570181141 percent lr= 0.001
 
epoch= 40  time= 0.7419912815093994  loss= 0.7683855692545573  val_loss= 1.638215959072113  f1= 0.36718591515763754  f2= 0.3882413410881827  f3= 

100%|██████████| 400/400 [00:01<00:00, 294.26it/s]


Start training with for 64 data points
Start computation of covariance matrix.
Covariance matrix of shape (64, 64)computed.
Start Cholesky decomposition of (64, 64) Matrix
Cholesky decomposition completed.
Create LHS of linear system for H at test points.
Creation of linear system completed.
Solve least square problem of dimension (129, 64)


...

100%|██████████| 50/50 [00:00<00:00, 541.51it/s]

100%|██████████| 50/50 [00:00<00:00, 2743.02it/s]

100%|██████████| 50/50 [00:00<00:00, 509.65it/s]

100%|██████████| 50/50 [00:00<00:00, 406.19it/s]

100%|██████████| 50/50 [00:00<00:00, 256.58it/s]

100%|██████████| 50/50 [00:00<00:00, 644.90it/s]

100%|██████████| 50/50 [00:00<00:00, 2205.74it/s]

100%|██████████| 34/34 [00:00<00:00, 586.77it/s]

100%|██████████| 50/50 [00:00<00:00, 312.46it/s]

100%|██████████| 50/50 [00:00<00:00, 567.65it/s]

100%|██████████| 50/50 [00:00<00:00, 2437.16it/s]

100%|██████████| 34/34 [00:00<00:00, 403.74it/s]

100%|██████████| 50/50 [00:00<00:00, 644.01it/s]

100%|██████████| 50/50 [00:00<00:00, 189.20it/s]

100%|██████████| 50/50 [00:00<00:00, 2788.84it/s]

100%|██████████| 13/13 [00:00<00:00, 258.37it/s]

100%|██████████| 50/50 [00:00<00:00, 239.33it/s]

100%|██████████| 50/50 [00:00<00:00, 2130.88it/s]

100%|██████████| 50/50 [00:00<00:00, 287.36it/s]

100%|██████████| 50/50 [00:00<00:00, 276.11it

 
epoch= 0  time= 0.2304074764251709  loss= 0.9687805735483402  val_loss= 1.0328126549720764  f1= 0.45757983064000524  f2= 0.4963525243247525  f3= 0.012775765588684722  f4= 0.0020724522925735385 percent lr= 0.001
 
epoch= 10  time= 2.9784164428710938  loss= 0.2414514175275477  val_loss= 0.18702128753066063  f1= 0.019003831956467526  f2= 0.17963919130898412  f3= 0.0002122617932202302  f4= 0.04259613026925338 percent lr= 0.001
 
epoch= 20  time= 5.639863014221191  loss= 0.07174519777706848  val_loss= 0.059400885412469503  f1= 0.003252700516459748  f2= 0.048312437109042504  f3= 0.00012687672709943963  f4= 0.02005318344920312 percent lr= 0.001
 
epoch= 30  time= 8.429654836654663  loss= 0.0565689377452633  val_loss= 0.045316766342148186  f1= 0.0023815561813542593  f2= 0.037728240535594065  f3= 0.00011731605279546728  f4= 0.016341824982840543 percent lr= 0.001
 
epoch= 40  time= 11.163262844085693  loss= 0.04622856833086145  val_loss= 0.03553447921294719  f1= 0.002266498687463908  f2= 0.030

100%|██████████| 400/400 [00:01<00:00, 325.42it/s]


Start training with for 1024 data points
Start computation of covariance matrix.
Covariance matrix of shape (1024, 1024)computed.
Start Cholesky decomposition of (1024, 1024) Matrix
Cholesky decomposition completed.
Create LHS of linear system for H at test points.
Creation of linear system completed.
Solve least square problem of dimension (2049, 1024)


100%|██████████| 400/400 [07:56<00:00,  1.19s/it]
